# Imports

In [3]:
import pandas as pd

# Load csv data

In [4]:
pd.read_csv('../data/train.csv')

,id,text,class,year
0,0,Licitació de la concessió administrativa de l'...,0,2013
1,1,Notificació de l'acord de deixar sense efecte ...,17,2013
2,2,Aprovació de l'expedient de dissolució i liqui...,17,2013
3,3,Aprovació inicial de la modificació puntual de...,16,2013
4,4,Aprovació definitiva de l'avanç del Pla de mil...,16,2013
...,...,...,...,...
216969,216969,Aprovació definitiva de la modificació de les ...,2,2020
216970,216970,Aprovació definitiva de l'acord de modificació...,2,2020
216971,216971,Aprovació de la modificació dels preus públics...,2,2020
216972,216972,Aprovació definitiva de les ordenances fiscals...,2,2020
